In [ ]:
import copy
from pathlib import Path
import warnings

import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import numpy as np
import pandas as pd
import torch
from lightning.pytorch.tuner import Tuner
from sklearn.preprocessing import StandardScaler
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import MultiNormalizer, EncoderNormalizer
from pytorch_forecasting.metrics import MAE, SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import (
    optimize_hyperparameters,
)

In [ ]:
warnings.filterwarnings("ignore", message=".*does not have valid feature names.*")

In [ ]:
data_path = "./rooms/F1_R1.csv"

max_encoder_length = 48
max_prediction_length = 5

lr = 3e-4
epochs = 5
batch_size = 128

In [ ]:
df = pd.read_csv(data_path)
df["hour"] = df["hour"].astype(str)
df["day"] = df["day"].astype(str)
df["month"] = df["month"].astype(str)
df = df.sort_values(["room_id", "timestamp"]).reset_index(drop=True)
df["time_idx"] = df.groupby("room_id").cumcount()
training_cutoff = df["time_idx"].max() - max_prediction_length

In [ ]:
train_dataset = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target=["humidity", "temperature", "co2", "electricity"],
    group_ids=["room_id"],
    min_encoder_length=max_encoder_length,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["room_id"],
    static_reals=["area", "num_windows", "window_area"],
    time_varying_known_categoricals=["hour", "day", "month"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["humidity", "temperature", "co2", "electricity"],
    target_normalizer=MultiNormalizer([
        EncoderNormalizer(method='standard',center=True,max_length=None,transformation=None,method_kwargs={}),
        EncoderNormalizer(method='standard',center=True,max_length=None,transformation=None,method_kwargs={}),
        EncoderNormalizer(method='standard',center=True,max_length=None,transformation=None,method_kwargs={}),
        EncoderNormalizer(method='standard',center=True,max_length=None,transformation=None,method_kwargs={})
    ]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)
train_dataloader = train_dataset.to_dataloader(train=True, batch_size=batch_size)

val_dataset = TimeSeriesDataSet.from_dataset(
    train_dataset, df, predict=True, stop_randomization=True
)
val_dataloader = val_dataset.to_dataloader(
    train=False, batch_size=batch_size * 10, num_workers=0
)

In [ ]:
x, y = next(iter(train_dataset.to_dataloader(batch_size=4)))
y[0]

In [ ]:
baseline_predictions = Baseline().predict(val_dataloader, return_y=True)

In [ ]:
targets = ["humidity", "temperature", "co2", "electricity"]
outputs = baseline_predictions.output
y = baseline_predictions.y

for i in range(len(targets)):
    print(f"{targets[i]}: {MAE()(outputs[i], y[0][i])}")

In [ ]:
model = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=lr,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    loss=MAE(),
    output_size=[1, 1, 1, 1],
    log_interval=10,
    reduce_on_plateau_patience=4,
)

print(f"Number of parameters in network: {model.size() / 1e3:.1f}k")

In [ ]:
trainer = pl.Trainer(
    accelerator="cpu",
    gradient_clip_val=0.1
)

In [ ]:
res = Tuner(trainer).lr_find(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

In [ ]:
early_stop_callback = EarlyStopping(
    monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
)
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

trainer = pl.Trainer(
    max_epochs=50,
    accelerator="cpu",
    enable_model_summary=True,
    gradient_clip_val=0.1,
    limit_train_batches=50,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

In [ ]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)